<a href="https://colab.research.google.com/github/enigma6174/tensorflow-learn/blob/develop/fundamentals/tensorflow_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Operations On Tensors

- Changing The Datatype
- Tensor Aggregation Functions
- Squeezing A Tensor
- One-hot Encoding
- Tensor Mathematics
- Numpy And Tensor Compatibility

In [151]:
import numpy as np
import tensorflow as tf

In [227]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [228]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [229]:
!nvidia-smi

Mon Dec 26 14:06:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    31W /  70W |    314MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [137]:
# helper function to generate a tensor of required shape
def generate_tensor(size, seed=42):
  return tf.cast(tf.math.multiply(tf.random.Generator.from_seed(seed).normal(shape=size), 100), dtype=tf.int32)

## [1] Changing The Datatype Of A Tensor

The datatype of a tensor can be changed with `tf.cast()`

In [ ]:
tensor = tf.constant([
    [10, 20, 30],
    [11, 13, 17]
])
tensor.dtype

tf.int32

In [4]:
# cahnge the datatype to float32
tensor = tf.cast(tensor, dtype=tf.float32)
tensor

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[10., 20., 30.],
       [11., 13., 17.]], dtype=float32)>

In [5]:
# change the datatype from float32 to float16 (reduced precision)
tensor = tf.cast(tensor, dtype=tf.float16)
tensor

<tf.Tensor: shape=(2, 3), dtype=float16, numpy=
array([[10., 20., 30.],
       [11., 13., 17.]], dtype=float16)>

In [6]:
# change the datatype from float16 to int64
tensor = tf.cast(tensor, dtype=tf.int64)
tensor

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[10, 20, 30],
       [11, 13, 17]])>

## [2] Tensor Aggregation Functions

> Condensing tensors from multiple values down to a smaller amount of values

In [24]:
# tensor of shape (4, 3) intialized with random values
tensor = generate_tensor((4, 3))
tensor

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ -75,   -6,    7],
       [-125,  -23, -181],
       [   9,  -50,  -75],
       [ -57,   14,  -23]], dtype=int32)>

In [25]:
# get the absolute values of the tensor
tf.math.abs(tensor)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ 75,   6,   7],
       [125,  23, 181],
       [  9,  50,  75],
       [ 57,  14,  23]], dtype=int32)>

### [2.a] Calculating The Mean

The mean of a tensor along any axis can be calculated using `tf.math.reduce_mean(tensor, axis)`. If the axis is not mentioned then by default, the entire tensor is reduced to a single mean

In [26]:
# calculate the mean of the tensor
tf.math.reduce_mean(tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=-48>

In [28]:
# calculate the mean along axis 0
tf.math.reduce_mean(tensor, axis=0)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([-62, -16, -68], dtype=int32)>

In [18]:
# calculate the mean along axis 1
tf.math.reduce_mean(tensor, axis=1)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ -24, -109,  -38,  -22], dtype=int32)>

In [29]:
# tensor of shape (3, 2, 3)
tensor = generate_tensor((3, 2, 3))
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>

In [30]:
# mean of the tensor (all dimensions reduced)
tf.math.reduce_mean(tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=-24>

In [31]:
# mean of the tensor along axis 0 
tf.math.reduce_mean(tensor, axis=0)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[-33,  -5, -57],
       [-20,  53, -82]], dtype=int32)>

In [32]:
# mean of the tensor along axis 1
tf.math.reduce_mean(tensor, axis=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[-100,  -14,  -87],
       [ -24,  -18,  -49],
       [  42,  104,  -74]], dtype=int32)>

In [33]:
# mean of the tensor along axis 2
tf.math.reduce_mean(tensor, axis=2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ -24, -109],
       [ -38,  -22],
       [ -33,   81]], dtype=int32)>

#### Implementation Of Mean Reduction Along 0-Axis

Consider the input tensor:  

```python
<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>
```  

The output for `tf.math.reduce_mean(tensor, axis=0)` is:  

```python
<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[-33,  -5, -57],
       [-20,  53, -82]], dtype=int32)>
```

In [98]:
# reate the initial tensor for tracking sum
t = tf.zeros(shape=(2, 3), dtype=tf.int32)

# add the tensors
for i in range(tensor.shape[0]):
  t = tf.math.add_n([t, tensor[i]])

# divide the tensor sums to get the mean 
tensor_0 = tf.cast(tf.math.divide(t, 3), dtype=tf.int32)
print(tensor_0)

tf.Tensor(
[[-33  -5 -57]
 [-20  53 -82]], shape=(2, 3), dtype=int32)


#### Implementation Of Mean Reduction Along 1-Axis

Consider the input tensor:  

```python
<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>
```  

The output for `tf.math.reduce_mean(tensor, axis=1)` is:  

```python
<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[-100,  -14,  -87],
       [ -24,  -18,  -49],
       [  42,  104,  -74]], dtype=int32)>
```

In [100]:
# implementation of reduce_mean along axis=1
tensor_1 = tf.zeros(shape=(1, 3), dtype=tf.int32)

for i in range(tensor.shape[0]):
  t = tf.zeros(shape=(3,), dtype=tf.int32)

  for j in range(tensor[i].shape[0]):
    t = tf.math.add_n([t, tensor[i, j, :]])

  # calculate the mean and reshape
  t_divided = tf.cast(tf.math.divide(t, tensor.shape[1]), dtype=tf.int32)
  t_reshape = tf.reshape(t_divided, [1, 3])

  # when i = 0 add the resultant tensor to initial tensor
  # otherwise concatenate resultant tensor
  if i == 0:
    tensor_1 = tf.math.add_n([tensor_1, t_reshape])
  else:  
    tensor_1 = tf.concat([tensor_1, t_reshape], axis=0)

print(tensor_1)

tf.Tensor(
[[-100  -14  -87]
 [ -24  -18  -49]
 [  42  104  -74]], shape=(3, 3), dtype=int32)


#### Implementation Of Mean Reduction Along 2-Axis

Consider the input tensor:  

```python
<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>
```  

The output for `tf.math.reduce_mean(tensor, axis=2)` is:  

```python
<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ -24, -109],
       [ -38,  -22],
       [ -33,   81]], dtype=int32)>
```

In [110]:
# implementation of reduce_mean along axis=2
tensor_2 = tf.zeros(shape=(1, 2), dtype=tf.int32)

for i in range(tensor.shape[0]):
  t = tf.zeros(shape=(2,), dtype=tf.int32)

  for j in range(tensor[i].shape[1]):
    t = tf.math.add_n([t, tensor[i, :, j]])

  # calculate the mean and reshape
  t_divided = tf.cast(tf.math.divide(t, tensor.shape[2]), dtype=tf.int32)
  t_reshape = tf.reshape(t_divided, [1, 2])

  # if i = 0 add resultant tensor to initial tensor
  # otherwise concatenate resultant tensor
  if i == 0:
    tensor_2 = tf.math.add_n([tensor_2, t_reshape])
  else:
    tensor_2 = tf.concat([tensor_2, t_reshape], axis=0)

print(tensor_2)

tf.Tensor(
[[ -24 -109]
 [ -38  -22]
 [ -33   81]], shape=(3, 2), dtype=int32)


### [2.b] Calculating The Sum

The sum of a tensor along any axis can be calculated using `tf.math.reduce_sum(tensor, axis)`. If the axis is not specified, by default the entire tensor is reduced to a single sum

In [111]:
# tensor of shape (4, 3) generated using random values
tensor = generate_tensor((4,3))
tensor

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ -75,   -6,    7],
       [-125,  -23, -181],
       [   9,  -50,  -75],
       [ -57,   14,  -23]], dtype=int32)>

In [112]:
# calculate the sum
tf.math.reduce_sum(tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=-585>

In [113]:
# calculate the sum along axis 0
tf.math.reduce_sum(tensor, axis=0)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([-248,  -65, -272], dtype=int32)>

In [114]:
# calcuate the sum along axis 1
tf.math.reduce_sum(tensor, axis=1)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ -74, -329, -116,  -66], dtype=int32)>

In [115]:
# tensor of shape (3, 2, 3) generated using random values
tensor = generate_tensor((3, 2, 3))
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>

In [116]:
# calculate the sum along axis 0
tf.math.reduce_sum(tensor, axis=0)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[-101,  -16, -173],
       [ -62,  159, -248]], dtype=int32)>

In [117]:
# calculate the sum along axis 1
tf.math.reduce_sum(tensor, axis=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[-200,  -29, -174],
       [ -48,  -36,  -98],
       [  85,  208, -149]], dtype=int32)>

In [118]:
# calculate the sum along axis 2
tf.math.reduce_sum(tensor, axis=2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ -74, -329],
       [-116,  -66],
       [-100,  244]], dtype=int32)>

### [2.c] Calculate The Maximum & Minumum

The maximum value of a tensor along any axis can be calculated using `tf.math.reduce_max(tensor, axis)`. Similarly, the minimum value of a tensor along any axis can be calculated using `tf.math.reduce_min(tensor, axis)`   

If the axis is not mentioned, the maximum across all dimensions is returned

In [119]:
# tensor of shape (4, 3) generated using random values
tensor = generate_tensor((4, 3))
tensor

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ -75,   -6,    7],
       [-125,  -23, -181],
       [   9,  -50,  -75],
       [ -57,   14,  -23]], dtype=int32)>

In [122]:
# maximum across all dimensions
tf.math.reduce_max(tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=14>

In [123]:
# maximum across axis 0
tf.math.reduce_max(tensor, axis=0)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 9, 14,  7], dtype=int32)>

In [124]:
# maximum across axis 1
tf.math.reduce_max(tensor, axis=1)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([  7, -23,   9,  14], dtype=int32)>

In [125]:
# tensor of shape (3, 2, 3) generated using random values
tensor = generate_tensor((3, 2, 3))
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>

In [126]:
# calculate the minimum across all axes
tf.math.reduce_min(tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=-181>

In [127]:
# calculate the minimum across axis 0
tf.math.reduce_min(tensor, axis=0)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ -75,  -50, -105],
       [-125,  -23, -181]], dtype=int32)>

In [128]:
# calcualte the minimum across axis 1
tf.math.reduce_min(tensor, axis=1)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[-125,  -23, -181],
       [ -57,  -50,  -75],
       [ -35,   40, -105]], dtype=int32)>

In [130]:
# calculate the minimum across axis 2
tf.math.reduce_min(tensor, axis=2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ -75, -181],
       [ -75,  -57],
       [-105,  -44]], dtype=int32)>

### [2.d] Calculating The Index Of Maximum And Minimum

To index of the maximum value across any axis of the tensor can be calculated using `tf.math.argmax(tensor, axis)` and the index of the minimum value across any axis of the tensor can be calculated using `tf.math.argmin(tensor, axis)` 

In [131]:
tensor = generate_tensor((4,3))
tensor

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ -75,   -6,    7],
       [-125,  -23, -181],
       [   9,  -50,  -75],
       [ -57,   14,  -23]], dtype=int32)>

In [132]:
# max value index
tf.math.argmax(tensor)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2, 3, 0])>

In [133]:
# max value index along axis 0
tf.math.argmax(tensor, axis=0)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2, 3, 0])>

In [134]:
# max value index along axis 1
tf.math.argmax(tensor, axis=1)

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([2, 1, 0, 1])>

In [138]:
tensor = generate_tensor((3,2,3))
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7],
        [-125,  -23, -181]],

       [[   9,  -50,  -75],
        [ -57,   14,  -23]],

       [[ -35,   40, -105],
        [ 120,  168,  -44]]], dtype=int32)>

In [141]:
# min value index across axis 0
tf.math.argmin(tensor, axis=0)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[0, 1, 2],
       [0, 0, 0]])>

In [142]:
# max value index across axis 0
tf.math.argmax(tensor, axis=0)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[1, 2, 0],
       [2, 2, 1]])>

In [143]:
# min value index across axis 1
tf.math.argmin(tensor, axis=1)

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[1, 1, 1],
       [1, 0, 0],
       [0, 0, 0]])>

In [144]:
# max value index across axis 1
tf.math.argmax(tensor, axis=1)

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[0, 0, 0],
       [0, 1, 1],
       [1, 1, 1]])>

In [147]:
# min value across axis 2
tf.math.argmin(tensor, axis=2)

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[0, 2],
       [2, 0],
       [2, 2]])>

In [146]:
# max value across axis 2
tf.math.argmax(tensor, axis=2)

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[2, 1],
       [0, 1],
       [1, 1]])>

### [2.e] Calculating The Variance And Standard Deviation

The variance of a tensor across the axes can be calculated by `tf.math.reduce_variance()` and the standard deviation of a tensor across the axes can be calculated by `tf.math.reduce_std()`  

If the axis is not mentioned, it will calculate across all the axes

In [154]:
# generate tensor with numpy random
tensor = tf.cast(tf.constant(np.random.randint(0, 100, size=50)), dtype=tf.float32)
tensor

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 9., 38.,  1., 83., 29., 37., 87., 93., 81., 91., 39., 10., 75.,
       64., 15.,  2., 12., 79., 52., 89., 73., 53., 45., 33., 35., 42.,
       99., 47., 89., 34., 25.,  7., 11., 12., 56.,  0., 93.,  2., 62.,
       71., 97., 74.,  3., 35., 10., 68., 20., 50., 34., 93.],
      dtype=float32)>

In [155]:
# calculate the variance
tf.math.reduce_variance(tensor)

<tf.Tensor: shape=(), dtype=float32, numpy=989.14764>

In [156]:
# calculate the standard deviation
tf.math.reduce_std(tensor)

<tf.Tensor: shape=(), dtype=float32, numpy=31.450718>

In [157]:
tensor = tf.cast(generate_tensor((4,3)), dtype=tf.float32)
tensor

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ -75.,   -6.,    7.],
       [-125.,  -23., -181.],
       [   9.,  -50.,  -75.],
       [ -57.,   14.,  -23.]], dtype=float32)>

In [158]:
# variance and standard deviation across axis 0
tf.math.reduce_variance(tensor, axis=0), tf.math.reduce_std(tensor, axis=0)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2301.    ,  551.1875, 5117.    ], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([47.968742, 23.477383, 71.53321 ], dtype=float32)>)

In [159]:
# variance and standard deviation across axis 1
tf.math.reduce_variance(tensor, axis=1), tf.math.reduce_std(tensor, axis=1)

(<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1294.8888, 4278.222 , 1240.2222,  840.6667], dtype=float32)>,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([35.984562, 65.40812 , 35.21679 , 28.994253], dtype=float32)>)

## [3] Squeezing A Tensor

Squeezing means to remove all dimensions of size 1 from a tensor

In [162]:
tensor = tf.constant(tf.random.uniform(shape=[1, 2, 3]))
tensor

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[0.31604695, 0.46567738, 0.40111053],
        [0.97965074, 0.44442737, 0.07780004]]], dtype=float32)>

In [163]:
# squeeze the tensor
tf.squeeze(tensor)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.31604695, 0.46567738, 0.40111053],
       [0.97965074, 0.44442737, 0.07780004]], dtype=float32)>

In [165]:
tensor = generate_tensor((3,1,3))
tensor

<tf.Tensor: shape=(3, 1, 3), dtype=int32, numpy=
array([[[ -75,   -6,    7]],

       [[-125,  -23, -181]],

       [[   9,  -50,  -75]]], dtype=int32)>

In [167]:
# squeeze the tensor
tf.squeeze(tensor)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ -75,   -6,    7],
       [-125,  -23, -181],
       [   9,  -50,  -75]], dtype=int32)>

## [4] One Hot Encoding

It is a form of representing categorical data with no mutual relationship among themeselves. A binary value is used to represent the categorical data. The value **1** represents the presence of said item and the value **0** represents the absence.

In [169]:
tensor = tf.constant(np.random.randint(0, 5, size=10))
tensor

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([3, 1, 4, 0, 2, 0, 0, 4, 2, 1])>

In [173]:
tf.one_hot(tensor, depth=5)

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

### [4.a] One Hot Encoding Of Labels

When the labels are not indices, some pre-processing is required to convert the labels to indices to be able to one-hot encode them

In [206]:
# array of labels
labels = np.array(['cat', 'dog', 'elephant', 'crocodile', 'dog', 'cat', 'dog', 'crocodile', 'elephant', 'dog', 'crocodile'])

# get unique value from labels
categories = np.unique(labels)

# convert the labels to integer representations
indices = tf.constant([i[0] for j in enumerate(labels) for i in enumerate(categories) if i[1] == j[1]])
indices

<tf.Tensor: shape=(11,), dtype=int32, numpy=array([0, 2, 3, 1, 2, 0, 2, 1, 3, 2, 1], dtype=int32)>

In [208]:
categories

array(['cat', 'crocodile', 'dog', 'elephant'], dtype='<U9')

In [207]:
# one hot encoding of the 
tf.one_hot(indices, depth=4)

<tf.Tensor: shape=(11, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

## [5] Mathematical Operations On Tensor

A quick look at more mathematical operations that can be performed on a tensor

In [210]:
# create a tensor using tf.range()
t = tf.range(5, 31, delta=5, dtype=tf.float32)
t

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([ 5., 10., 15., 20., 25., 30.], dtype=float32)>

In [212]:
# square of tensor
tf.math.square(t)

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([ 25., 100., 225., 400., 625., 900.], dtype=float32)>

In [213]:
# squre root of tensor
tf.math.sqrt(t)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([2.2360678, 3.1622777, 3.8729832, 4.4721355, 5.       , 5.4772253],
      dtype=float32)>

In [214]:
# square root of tensor
tf.math.sqrt(tf.math.square(t))

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([ 5., 10., 15., 20., 25., 30.], dtype=float32)>

In [215]:
# natural logarithm (ln x) of tensor
tf.math.log(t)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([1.609438 , 2.3025851, 2.7080503, 2.9957323, 3.218876 , 3.4011974],
      dtype=float32)>

## [6] Tensors And Numpy

The entire numpy library is based around **arrays** whereas the entire TensorFlow library is based around the concept of **tensors**. The key difference between a **numpy array** and a **tensor** is that tensors can run on a GPU whereas numpy arrays cannot.  

There is a high degree of compatibility between numpy arrays and tensors

In [217]:
# create a tensor from numpy array
tensor = tf.constant(np.random.randint(0, 20, size=5))
tensor

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([10,  8, 14,  1,  7])>

In [219]:
# convert tensor to numpy array
array = np.array(tensor)
array, type(array)

(array([10,  8, 14,  1,  7]), numpy.ndarray)